<center>
    <b><h1> Couserwork Assignment: Text Classification</h1></b>
</center>

<center>
    <h3>University of London</h3>
</center>

<center>
    <h3>BSc in Computer Science</h3>
</center>

<center>
    <h3>Natural Language Processing</h3>
</center>

<div style="page-break-after: always;"></div>

<h1> Table of Content</h1>

<style>
ul {
    font-size: 20px;
}

ul ul {
    font-size: 15px;
}
</style>

<ul>
    <li>
        <a href="#intro">1. Introduction</a>
        <ul>
            <li><a href="#1.1">1.1 Problem Area</a></li>
            <li><a href="#1.2">1.2 Objectives</a></li>
            <li><a href="#1.3">1.3 Choice of Dataset</a></li>
        </ul>
    </li>
    <li>
        <a href="#implementation">2. Implementation</a>
        <ul>
            <li><a href="#2.1">2.1 Processing</a></li>
            <ul>
                <li><a href="#2.1.1">2.1.1 Data Extraction and Cleaning</a></li>
                <li><a href="#2.1.2">2.1.2 Prelimary Data Exploration and Data Cleaning</a></li>
                <li><a href="#2.1.3">2.1.3 Vocabulary statistic</a></li>
            </ul>
            <li><a href="#2.2">2.2 Baseline performance</a></li>
        </ul>
    </li>
    <li>
        <a href="#Conc">3. Conclusion</a>
        <ul>
            <li><a href="#3.1">3.1 Evaluation</a></li>
            <li><a href="#3.2">3.2 Project Evaluation and Summary</a></li>
        </ul>
    </li>
</ul>


<div style="page-break-after: always;"></div>

<h2 id="intro">1. Introduction</h2>
<h3 id="1.1">1.1 Problem area</h3>

Our current generation has witnessed unprecedented global growth, be it in the economy or technology sectors. However, since the late 2010s, the landscape has undergone significant transformations. The conflicts around the world, deceleration in multiple sectors, and rise of protectionism have posed challenges to the old strategy of relying solely on continuous economic expansion. This challenge is fuelled by the unfavorable economic outlook observed since the 2020s. As a result, individuals have become more conservative and cautious in their spending habits. Consequently, businesses across sectors must vigilantly monitor shifts in consumer sentiment. By extracting insights from sentiment analysis, companies can understand consumer attitudes toward their products and services, as well as their overall brand image. This knowledge enables companies to promptly adapt and implement new strategies, ensuring their relevance in this volatile economic environment.

The film industry has been severely impacted by declining attendance, attributed to both slowing economic growth and a lack of compelling content. (sg) The coronavirus pandemic has further exacerbated the situation with lockdowns and admission restrictions. (us) Movie theaters now face fierce competition from alternative entertainment sources, especially video games, which experienced an impressive 20% growth in 2020. (us) To ensure continued prosperity, industry decision-makers must promptly reassess their strategies, giving paramount importance to audience opinions and feedback. Sentiment analysis becomes a critical tool for understanding audience preferences and sentiments, enabling the film industry to adapt and thrive in an ever-evolving entertainment landscape.

Despite the prevailing economic challenges, the film industry achieved a remarkable milestone in 2019, surpassing the $101 billion USD mark. This multi-billion-dollar industry still holds significant potential for prosperity. While attracting new consumers to choose movies as their primary source of entertainment may present difficulties, companies can focus on retaining their existing viewer base. Leveraging sentiment analysis, specifically tailored to movie reviews, becomes essential as it provides valuable insights into viewers' preferences and dislikes. By incorporating these insights, future movie productions can effectively cater to the preferences of the target audience, fostering continued success in the industry.

Factually speaking, the internet has become a vast database of opinions created by consumers. They express their thoughts through various channels such as Twitter, Facebook, IMDB, and Rotten Tomatoes. The model developed for sentimental analysis for the film industry can be transferred to and utilized in other sectors of the economy.  With appropriate refinement, this model has the potential to provide valuable insights into user sentiments in domains that may not have direct connections to the film industry. This demonstrates the versatility and applicability of sentiment analysis beyond its initial scope, offering a valuable tool for understanding consumer sentiment in diverse fields.

Word Count:436

<h3 id="1.2">1.2 Objective</h3>

The <b>objective</b> of this research is to conduct sentiment analysis on movie reviews utilizing text classification models. The project's focus will be on implementing and comparing the effectiveness of different classification algorithms. 

The <b>aim</b> of this project will be categorizing users' movie reviews into binary classes of positive or negative sentiments. 

This research aims to contribute to the field of sentiment analysis by adding to the knowledge of users' sentiment towards movies and using this knowledge to provide valuable insights for the film industry.

While it is true that there have been many researches conducted on sentiment analysis and its application in the film industry, this particular research still holds its value and relevance. Granted that this is not groundbreaking research in this sector, it still <b>contributes to the existing body of knowledge</b> and <b>provides more insights</b>, and has the potential to be applied elsewhere. 

The insights are essential to <b>filmmakers, production companies</b>, and other stakeholders such as <b>investors and actors/actresses</b> to make informed decisions with regard to content creation, marketing, investments, and engagement with the audience.

The result of this research can be implemented further in other sectors as a form of <b>Transfer Learning</b>. For instance, it can be used to examine the positive and negative sentiment in other reviews such as music reviews and Application stores reviews. 

The implementation of this research will consist of the following parts:

<b>Data Cleaning and Processing</b>: 
This section aims to obtain the data and prepare it for further analysis. The data will need to be cleaned, consistent, and balanced at the end. It will identify and remove irrelevant features while focusing on important ones. 

<b>Simple Textual Analysis</b>: Simple analysis without the use of machine learning and neural networks will be conducted. It would include analysis such as Lexical analysis and word frequency distribution.

<b>Building Classfication Models</b>:  
Multiple Machine Learning Classification models will be constructed and trained on the processed data. These models aim to categorize the comments on the movies into positive and negative sentiments, enabling the analysis of user opinions. 

<b>Performance Evaluation</b>: The models constructed will undergo evaluations and comparisons using metrics such as accuracy, recall, and F1-score, together with the baseline model. The best-performing model will be identified by its ability to accurately classify the comments into positive and negative sentiments.

<b>Conclusion</b>: Final findings will be concluded from two perspectives, building the model for future use (Performance Evaluation) and analysis of current data (Simple Textual Analysis). It allows more informed conclusions regarding sentiment analysis and its applicability to movie comments.


Word Count: 420

<h3 id="1.3">1.3 Choice of Dataset</h3>

It is essential to choose a good choice of data. A range of data from many websites was considered, including the Rotten Tomatoes movies and critic reviews dataset and Top Popular Movie Review Data Set from Kaggle. Ultimately, the Large Movie Review Dataset was chosen due to its large size. 

The dataset is available to download from [here](https://ai.stanford.edu/~amaas/data/sentiment/). It is in the compressed tar format. It contains movie reviews with their associated binary sentiment labels. The dataset contains 50,000 reviews split into 25k for training and 25k for testing. This data set is a balanced dataset with an equal number of contains labeled as positive and negative.

The data is stored in a text document in two folders, one for 'pos', and another for 'neg'. Each document's name will contain its unique ID and the corresponding rating. As the data is not stored in the traditional CSV format, an algorithm will be needed to extract the information for the folder.

The data is obtained from the Stanford Artificial Intelligence Laboratory, a laboratory at Stanford University. This a credible data that has also been widely used in research for many years. Although Dr. Andrew Maas, the first author of this dataset, did not explicitly specify the term of usage, the dataset is publicly available from the school's website.

From an ethical point of view, the data did not include personal information about the creator of the review, making this have a high level of anonymity. Yet, a link is provided for every unique ID in the data, linking to the user comment page of the movie. This practice ensures the user remains largely anonymous while providing clues for data analysts to trace back to the movies if required.

In summary, thorough research is conducted to find the Large Movie Review Dataset due to its large size and availability. This makes it an ideal source for sentimental analysis as it offers a substantial amount of diverse data with labels. 

Word Count 327

<h3 id="1.4">1.4 Evaluation Methodology</h3>

To effectively assess the performance of the models in this project Evaluation metrics such as accuracy, presision, recall and F1 score will be used. The confusion matrix will also be analysed. However, the main focus will be on comparing the accuracy across the models and the baseline. 

With reference to the formula below, accuracy measures the overall correctness of the classification model by calculating the correctly classfied samples over all of the samples

$$
 \text{Accuracy} = \frac{\text{True Positives} + \text{True Negatives}}{\text{Total Samples}}
 $$

Precision, on the other hand, only considers those which are correctly classified as positive.

$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
$$

Recall finds those which are correctly labbeled as positive among those which are actually positive.

$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
$$

F1 score provides a balanced measure of the model's performance by combining both precision and recall. It considers the model's ability to correctly identify positive instances.

$$
\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} 
$$

Confusion matrix provides the summary of the overall performance of the model by displaying the number for true positives, true negatives, false positives, and false negatives. It is a detailed breakdown of the models prediction against the actual sentiment.

In this study, accuracy will be the primary evaluation metric as it provides a balanced consideration of both positive and negative sentiments. However, during the optimization of the models, other metrics such as precision, recall, and F1-score may also be considered. Nevertheless, for the purpose of evaluation across all models, accuracy will be the sole metric utilized.

Word Count: 246

<h2 id="implementation">2. Implementation</h2>
<h3 id="2.1">2.1 Processing</h3>
<h3 id="2.1.1">2.1.1 Data extraction</h3>

Before implementing the machine learning algorithm for natural language processing, the Large Movie Review Dataset obtained from the Stanford Artificial Intelligence Laboratory will need to be extracted and cleaned. The libraries for the entire analysis are imported together first, however, only some libraries, such as os library for managing the system, random library for selecting a random sample of data and pandas for storing and handling data frames will be used in the extraction part.

In [ ]:
# Import library needed for data analysis

# os is used for handling the system
import os

# random for randomly selecting the data
import random

# pandas for storing and handling the data in data frame
import pandas as pd

# matplotlib for visualising the data
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Regular expression handling
import re

# Numpy for calculation
import numpy as np

# NLTK for natural language processing
import nltk
from nltk.corpus import stopwords #stop words removal
from nltk.stem import WordNetLemmatizer #replace words with similar meaning
from nltk.tokenize import word_tokenize #tokenizing the string
from nltk import ne_chunk
from nltk.tree import Tree
from nltk.collocations import BigramCollocationFinder #collation calculation
from nltk.metrics import BigramAssocMeasures #collation calculation
from nltk.probability import FreqDist #distribution calculation

# spliting of data into test and train dataset
from sklearn.model_selection import train_test_split

# RNN
from tensorflow.keras.preprocessing.text import Tokenizer #tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences #padding
from tensorflow.keras.models import Sequential #deeplearning model
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dropout, Dense #deeplearning layers
from tensorflow.keras import regularizers # regularisation
from tensorflow.keras.models import load_model # loading model for testingS

# Navie Bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# SVM
from sklearn.svm import SVC

# BoW
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Logistic regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

# comparison
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Bert
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error

# wordcould display
from wordcloud import WordCloud

In this stage, data will be extracted and stored locally. The location of the data file is first specified to extract the data

The entire dataset contains <b>50,000 data entries</b>, a <b>balanced split of 25,000 entires</b> are in the testing and training file. To speed up the development process, only one of the folders will be used. In the folder selected, a randome selection of files will conducted again to reduce the datasize to 10,000 (5000 positive, 5000 negative). 

As this dataset is quite widely explored globally, there are advanced models such as pretrained tensor models developed based on this dataset. This report will thus attempt to <b>deploy a published pretrained model</b>. As these models are likely to be trained on data the train folder of the original dataset, this report will <b>use the test folder from original dataset</b> instead for both testing and training. 

In [ ]:
# Path to the folders
pos_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\test\pos'
neg_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\test\neg'

<b>Format of data:</b> As the data obtained is in the form of <b>text document</b> (.txt file) stored in positive and negative sentiment folder with each individual file name as the rating and the ID, a file counter is needed to check for the length of the data.

In [ ]:
# Helper functions;

# Count the number of files in the data folder
def count_files(folder_path):
    if not os.path.isdir(folder_path):
        print(f"Error: {folder_path} is not a valid directory.")
        return

    file_count = 0

    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

Using the function as define above, the number of the files in both positive and negative folder are obtained: 

In [ ]:
# Positive data file avaliable
file_count = count_files(pos_folder)
print(f"The number of files in positive data folder is: {file_count}")

In [ ]:
# Negative data file avaliable 
file_count = count_files(neg_folder)
print(f"The number of files in nagtive data folder is: {file_count}")

As this training data set is <b>too big</b> for the computer, only <b>5000 data</b> from positive and negative folder respectively will be selected to make the program runs faster. Then these data will be further seperated into test and training sets. These data will be selected using the random library.

Information such as the ID of the comment and the rating of the comment will be extracted from the file name using "_" as the separator, while the actual comment will be extracted by reading the file content.

Some helper functions are defined to extract the information and data from the data folder:

In [ ]:
# Helper functions

# Function to extract unique ID and rating from file name
def extract_info(file_name):
    id_rating = file_name.split('.')[0]
    unique_id, rating = id_rating.split('_')
    return int(unique_id), int(rating)

# Function to extract the data from the file based on the aclImdb data structure
def extract_files(folder, num_files, sentiment):   
    files = random.sample(os.listdir(folder), num_files)
    data = pd.DataFrame(columns=['Sentiment', 'Unique ID', 'Rating', 'Content'])
    
    for file_name in files:
        file_path = os.path.join(folder, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read() # read content for the review
        unique_id, rating = extract_info(file_name)  # get ID and rating from file name
        file_data = pd.DataFrame({'Sentiment': [sentiment], 'Unique ID': [unique_id], 'Rating': [rating], 'Content': [content]})
        data = pd.concat([data, file_data], ignore_index=True)
    
    return data

The following code randomly extract 5000 random data from the folder using the helper function defined above:

In [ ]:
# Extracting data
pos_data = extract_files(pos_folder, 5000, 'pos')
neg_data = extract_files(neg_folder, 5000, 'neg')

# Print the resulting DataFrame
raw_data = pd.concat([pos_data, neg_data], ignore_index=True)

# Display the start of the data
raw_data.head()

The raw data obtained from the dataset is thus extracted, however, cleaning is needed before extracting useful knowledge from the content.

<br>

<h3 id="2.1.2">2.1.2 Prelimary Data Exploration and Data Cleaning</h3>

In [ ]:
# Examining the data
raw_data.describe()

It was noted that both Unique ID and Content has duplicated values after the extraction as the frequency of the data is not 1. 

However, it is <b>not ideal</b> for the model to take in repeated reading in the content field as it does not add any more knowlegde to the model. Moreover, the original unique ID also do not provide much information after combining the data from both positive sentiment and negative sentiment.

Thus, an algorithm will be implemented to check for these values and handle them.

<b>For duplicates in Unique ID</b>, as a <b>new ID will be assigned</b> as the index for the data frame, there is <b>no need</b> for the Unique ID from positive sentiment and negative sentiment folder. Thus, the <b>unique ID column will be dropped</b>. 

<b>For duplicates in content</b>, it will first find the duplicates, then remove the <b>first instance</b> of the duplicates. To ensure the overall dataset remain balanced, it will perform the random selection of data from the original data file again. These two functions will be <b>called recursively</b> until there is <b>no repetition</b> in the dataframe. The cleaned data will be saved in a new data frame.

<h4>Removing duplicates and old Unique ID</h4>

The following code <b>removes</b> the unique ID from the data:

In [ ]:
# Remove unique ID column from the data
org_ID_removed_data = raw_data.drop("Unique ID", axis=1)

The following function <b>finds</b> the dupicates and <b>returns</b> the duplicated pairs:

In [ ]:
# Helper function
# Check for repetetion of data
def find_duplicates(dataframe, column):
    duplicates = dataframe[dataframe[column].duplicated(keep=False)]
    duplicates_sorted = duplicates.sort_values(by=column)

    if duplicates_sorted.empty:
        print("No repeated data entry found.")
    else:
        print("Repeated data entry(ies) found in "+str(column))
        duplicates_pairs = duplicates_sorted.groupby(column).apply(lambda x: x.reset_index(drop=True))
        duplicates_pairs.reset_index(drop=True, inplace=True)
        return duplicates_pairs

The following function <b>removes the duplicated data</b> and <b>add in data of the same size</b> from the positive or negative folder inorder to make the dataframe balanced:

In [ ]:
# Helper function
# replace duplicates with other values
def get_balance_data_with_duplicates(duplicate_content, original_data, pos_folder, neg_folder):
    # Remove the first occurrence of unique items in the duplicated content
    data_to_remove = duplicate_content.drop_duplicates(subset='Content', keep='first')
    org_ID_removed_data = original_data.drop_duplicates(subset='Content', keep='first')

    # Count the number of pos and neg entries in 'data_to_remove'
    pos_count = data_to_remove[data_to_remove['Sentiment'] == 'pos'].shape[0]
    neg_count = data_to_remove[data_to_remove['Sentiment'] == 'neg'].shape[0]

    # Add the removed duplicates such that the data is balanced
    added_pos_data = extract_files(pos_folder, pos_count, 'pos')
    added_neg_data = extract_files(neg_folder, neg_count, 'neg')

    # Remove the 'Unique ID' column from added data
    added_pos_data.drop("Unique ID", axis=1, inplace=True)
    added_neg_data.drop("Unique ID", axis=1, inplace=True)

    added_data = pd.concat([added_pos_data, added_neg_data], ignore_index=True)
    balanced_data = pd.concat([org_ID_removed_data, added_data], ignore_index=True)

    return balanced_data

As there could still be duplicates present in the new files that are added, a recursive algorithm is used to check and replace the replicate.

The following function will call get_balance_data_with_duplicates and find_duplicates functions will be called to check and replace the duplicated data until there is no more repeated entries:

In [ ]:
# recursively call the previouse two function untill data is balanced and no repetition
def recursive_data_balancing(dataframe, column, pos_folder, neg_folder):
    duplicates = find_duplicates(dataframe, column)

    if duplicates is None:
        print("No more duplicates found.")
        return dataframe

    # Call get_balance_data_with_duplicates with the duplicate content
    balanced_data = get_balance_data_with_duplicates(duplicates, dataframe, pos_folder, neg_folder)

    # Recursively check for duplicates
    print("Recursive call to balance the data.")
    return recursive_data_balancing(balanced_data, column, pos_folder, neg_folder)

Now, the last recursive function is called with the data extracted fed into the function. It will return a balanced dataset of positive and negative data with no repeatition:

In [ ]:
balanced_data = recursive_data_balancing(org_ID_removed_data, 'Content', pos_folder, neg_folder)

The algorithm and outputs confirms that a balanced dataset with no repetition was extracted from the data downloaded from the Stanford University AI Lab

<h4>Simple statistic analysis</h4>

Besides the ensure balancing and no repetition of data, some other checks were also performed on the dataset

In [ ]:
# checking the overall information
balanced_data.info()

In [ ]:
# double checking the total number of data entries
print("Number of entries in 'Sentiment' column:", balanced_data['Sentiment'].shape[0])
print("Number of entries in 'Rating' column:", balanced_data['Rating'].shape[0])
print("Number of entries in 'Content' column:", balanced_data['Content'].shape[0])

In [ ]:
# check for null data
balanced_data.isnull().sum()

The dataset extracted contains <b>3 columns</b> with <b>10000 rows</b>. There is <b>no null data</b> in all the data entries

In [ ]:
# double checking for unique data
balanced_data.nunique()

This section confirms that the recursive algorithm above is working to remove duplicate while ensuring a balanced data. There is repetition in the Content column in the data set.

The entries for the other two columns are examined below:

In [ ]:
balanced_data['Sentiment'].value_counts()

In [ ]:
balanced_data['Rating'].value_counts()

This shows that there is a <b>balanced mix</b> of positive (pos) and negative (neg) data. The Rating is <b>between 1 to 4 and 7 to 10</b>. According the information in the readme file attached in the dataset downloaded, entries with rating <b>5 and 6</b> are removed.

A <b>distribution</b> of the Rating in the form of <b>bar graph</b> and the propotion of each sentiment in the form of a <b>pie chart</b> is plotted for visualisation:

In [ ]:
# Distribution of Ratings
rating_counts = balanced_data['Rating'].value_counts().sort_index()

# Distribution of Sentiment
sentiment_counts = balanced_data['Sentiment'].value_counts()

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot Distribution of Ratings
ax1.set_xticks(range(1, 11))
ax1.set_xticklabels(range(1, 11))
ax1.bar(rating_counts.index, rating_counts.values)
ax1.set_xlabel('Rating', fontsize=12)
ax1.set_ylabel('Count',  fontsize=12)
ax1.set_title('Distribution of Ratings')

# Plot Distribution of Sentiment
ax2.pie(sentiment_counts.values, 
        labels=None, 
        autopct='%1.1f%%', 
        startangle=90) # no labels, set as 1 decimal place, title the angle by 90 degree
ax2.set_title('Distribution of Sentiment')
ax2.legend(sentiment_counts.index, loc="best", fontsize=12)

# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()

It can be seen that this dataframe contains 10000 with a <b>balanced distribution</b> of positive and negative sentiment data. The <b>most common rating</b> is 1 followed by 10. Both of them are the most frequent term in their sentiment respectively.

There is also <b>no null data</b> in all three columns.

<h4>Collation analysis:</h4>

<b>Collation</b> is obtained using BigramAssocMeasures from the NLTK library to analyze the relationship between nearby texts. This is useful to <b>identify negations</b>, which can pose challenges in the sentimental analysis when tokenizing data into n-grams.

For simplicity, the analysis will only focus on the negations for bigrams.

In [ ]:
# Function to calculate collocations in a given text
def calculate_collocations(text):
    words = nltk.word_tokenize(text)
    finder = BigramCollocationFinder.from_words(words)
    finder.apply_freq_filter(2)  # Adjust the frequency threshold as needed
    collocations = finder.nbest(BigramAssocMeasures.pmi, 10)  
    return collocations

# Combine the content of balanced_data['Content'] into a single string
text = ' '.join(balanced_data['Content'])

# Calculate collocations for the combined text
collocations = calculate_collocations(text)

# Create a DataFrame to store the collocations and frequencies
collocations_df = pd.DataFrame(collocations, columns=['Word 1', 'Word 2'])
collocations_df['Frequency'] = collocations_df.apply(lambda row: text.count(' '.join(row)), axis=1)

# Sort the DataFrame by frequency in descending order
collocations_df = collocations_df.sort_values('Frequency', ascending=False).reset_index(drop=True)

The result is displayed below:

In [ ]:
# Display the top 5 collocations with frequencies
collocations_df.head(10)

Based on the analysis above, there <b>does not</b> seem to be any negations as there is no common occurance of terms such as "not good" or "not nice".

<h4>Frequency analysis:</h4>

Next, the entries were tokenized to examine the common words used in the reviews using frequency distribution:

In [ ]:
# Tokenize the words in the balanced_data["Content"] column
all_words = nltk.word_tokenize(' '.join(balanced_data["Content"]))

# Calculate the frequency distribution of words
freq_dist = FreqDist(all_words)

# Get the 30 most common words
most_common_words = freq_dist.most_common(30)

# Extract the words and their frequencies
words, frequencies = zip(*most_common_words)

# Plot the number of occurrences
plt.figure(figsize=(10, 6))
plt.bar(words, frequencies)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 30 Most Common Words')
plt.xticks(rotation=45)
plt.show()

It was noted that most of the common words were <b>punctuation marks</b> (such as ",", ".", "<" and ">"), <b>determiners</b> (such as "the", "that", and "this") as well as <b>conjunctions</b> (such as "and", "for"). Thus a stopwords and punctuation removal is needed to obtain the part of the string that contains the most information. This will be done using the stopwords and Punkt tokenizer, which is a pre-trained sentence tokenizer available in NLTK.

Other common words that are present in the reviews are <b>related to the subject domain</b> (movie review. Some of the examples of these words are "movie" and "film".

It is also noted that besides the common stopwords and words that is related to the subject domain(moview review), there is also another commonly used term <b>"br"</b>. Further analysis of this term will thus be carried out.

<h4>lexical analysis:</h4>

Lexical Diversity is calculated using the equation below. It measure of how diverse the vocabulary used in the positive and negative reviews by examining the ratio of unique words to the total number of words in the text.

$$
\text{Lexical Diversity} = \frac{\text{Number of Unique Words}}{\text{Total Number of Words}}
$$


In [ ]:
# Separate the data based on sentiment for balanced data
positive_data = balanced_data[balanced_data['Sentiment'] == 'pos']
negative_data = balanced_data[balanced_data['Sentiment'] == 'neg']

In [ ]:
# Helper function to calculate the lexical diversity
def calculate_lexical_diversity(text):
    all_words = nltk.word_tokenize(text)
    total_words = len(all_words)
    unique_words = len(set(all_words))
    lexical_diversity = unique_words / total_words
    return lexical_diversity

In [ ]:
pos_lexical_diversity = calculate_lexical_diversity(' '.join(positive_data["Content"]))
neg_lexical_diversity = calculate_lexical_diversity(' '.join(negative_data["Content"]))

print("Lexical Diversity before cleaning (Positive Sentiment):", pos_lexical_diversity)
print("Lexical Diversity before cleaning (Negative Sentiment):", neg_lexical_diversity)

<h4>Cleaning for the use of tags</h4>

Firstly, the data entries with term br is filtered out and displayed as below:

In [ ]:
# Filter the entries containing "br"
filtered_data = balanced_data[balanced_data["Content"].str.contains("br")]

# Create a copy of the filtered DataFrame
filtered_data = filtered_data.copy()

# Extract the substring with 10 characters before and after "br"
filtered_data.loc[:, "Snippet"] = filtered_data["Content"].str.extract(r"(.{0,10}br.{0,10})")

# Display the filtered data
filtered_data[["Snippet"]]

It is noticed that the dataset uses <b> &lt;br /&gt; tag </b> as a part of the content. This sytex is mostly used to symbolise a change of line. Thus, the &lt;br /&gt; will be <b>removed</b> as part of the stopwords removal. 

To prevent other issues related to tags, a further check is conducted to verify other usage of < /> and <> using <b>regular expression</b>.

In [ ]:
# Check for other uses of tags using the term "< />"
# Filter the entries containing "<" and "/>"
data_tags = balanced_data[balanced_data["Content"].str.contains("<.*?/>")]

# Create a copy of the tags
data_tags = data_tags.copy()

# Extracting the substrings starts with "<" and end with "/>"
data_tags["tags"] = data_tags["Content"].str.extract(r"(<.*?/>)")

# Get the unique entries of tags
tags = data_tags["tags"].unique()

# Print the unique entries
for tag in tags:
    print(tag)

The above algorithm confirms that the <b>only tag used</b> is &lt;br /&gt;

Thus, the data need to take not of the use of  &lt;br /&gt; when carry out stopwords removal

The frequency of the symbols for tags are also checked:

In [ ]:
# Get the occurrences of tag markings
punctuation_occurrences = {
    "/": freq_dist["/"],
    "<": freq_dist["<"],
    ">": freq_dist[">"]
}

print("Occurrences of specific punctuation marks:")
for punctuation, count in punctuation_occurrences.items():
    print(f"{punctuation}: {count}")

From the algorithm above, it was noticed that the use of < and > were largely the same while the use of / is slightly higher.

This could indicate <b>other usage</b> of "/" besides tags.

Thus, the entire data is filtered again using regular expression which categorises every possible conbinations of the uses of "/" such as number / number and number / special characters.

In [ ]:
# Checking other uses
content_column = balanced_data['Content']

# Define the categories
categories = {
    'num/num': r'(?P<before>\d+(\.\d+)?)/(?P<after>\d+(\.\d+)?)',
    'num/special': r'(?P<before>\d+(\.\d+)?)/(?P<after>[\W\t])',
    'special/num': r'(?P<before>[\W\t])/(?P<after>\d+(\.\d+)?)',
    'num/alpha': r'(?P<before>\d+(\.\d+)?)/(?P<after>[a-zA-Z])',
    'alpha/num': r'(?P<before>[a-zA-Z])/(?P<after>\d+(\.\d+)?)',
    'alpha/special': r'(?P<before>[a-zA-Z])/(?P<after>[^\w\s\t])',
    'special/alpha': r'(?P<before>[^\w\s\t])/(?P<after>[a-zA-Z\s])',
    'alpha/alpha': r'(?P<before>[a-zA-Z])/(?P<after>[a-zA-Z])',
    'special/special': r'(?P<before>[^\w\s\t])/(?P<after>[^\w\s\t])',
    'space/alpha': r'(?P<before>\s)/(?P<after>[a-zA-Z])',
    'alpha/space': r'(?P<before>[a-zA-Z])/(?P<after>\s)',
    'space/space': r'(?P<before>[\s\t])/(?P<after>[\s\t])'
}

# Categorize the strings and extract characters before and after "/"
results = []
for category, pattern in categories.items():
    matches = content_column.str.extractall(pattern)
    if not matches.empty:
        matches = matches.reset_index()
        matches = matches.rename(columns={'level_0': 'index'})
        matches = matches[['index', 'before', 'after']]
        matches['category'] = category
        results.append(matches)

# Combine the results into a single dataframe
if results:
    combined_results = pd.concat(results)
    combined_results = combined_results.reset_index(drop=True)  # Reset the index
else:
    combined_results = pd.DataFrame()


The frequency of the isage can be determined using the code below:

In [ ]:
category_counts = combined_results['category'].value_counts().reset_index()
category_counts.columns = ['category', 'count']
category_counts = category_counts.sort_values('count', ascending=False)
print(category_counts)

The most common usage of the "/" is examined and shown below:

In [ ]:
a_a_df = combined_results[combined_results['category'] == 'alpha/alpha'].copy()
a_a_df.head(5)

In [ ]:
num_num_df = combined_results[combined_results['category'] == 'num/num'].copy()
num_num_df.head(5)

With that in mind, the original data was re-examined and realised the most common usage of"/" is for <b>choices between two words</b> (Eg. June/July), while the second most common usage is <b>two numbers</b> to show a rating (Eg. 8.5/10). 

Thus, when carrying out text processing pipelines later, <b>&lt;br /&gt; will be removed </b>as it is tag that provides little insights about the text. Moreover, numbers around "/" in the form of <b>number / number</b> will also be <b>removed</b> as it could provide information about the rating given by the viewer which should not be seen by the model.

A column in the dataframe is created to store the data to be removed:

In [ ]:
num_num_df['string_removal'] = num_num_df['before'] + '/' + num_num_df['after']
num_num_df

The data is now ready to be tokenized and transformed into a simpler term for a more detailed Natural Language Processing analysis.

<h4>Stopword removal and lemmatization</h4>

Stopwords removal and lemmatization are two important steps in Natural language processing. 

Stop words removal removes the commonly occured words that does not have significant contribution to the meaning of the text, such as "a", "the","is". This process reduces noise and shortens the input text size. The overall unique number of words decreases, thus lexical diversity will also decreases.

Lemmatization reduces the words into there base or root form. The variations in words will thus decrease, also result in a <b>decrease in lexical diversity</b>. 

The list of words will be downloaded from NLTK library to carry out this process:

In [ ]:
# Download NLTK resources
nltk.download('stopwords', quiet=True) # Stop words removal
nltk.download('wordnet', quiet=True) # Lemmatization
nltk.download('words', quiet=True) # Spelling check
nltk.download('maxent_ne_chunker', quiet=True) # Expand short forms

Some functions are needed to carry out the stop word removal and lemmatization. As the effects of every steps are evaluated, these steps could be called more than once. Thus, all of them are in the form of functions:

In [ ]:
# Helper function
# To change short forms to full forms
def expand_short_forms(word, previous_word):
    if word == "'s":
        # Check if the previous word is a proper noun or named entity
        if previous_word[0].isupper() or isinstance(ne_chunk([(previous_word, '')]), Tree):
            # Likely indicating possession
            return word
        else:
            # Likely contraction of "is"
            return "is"
    elif word == "n't":
        return "not"
    else:
        return word

The actual code for text processing pipelines are as follows:

In [ ]:
# Helper function
# Remove specific strings identified
def remove_string_entries(text, string):
    if string is not None:
        text = text.replace(string, '')
    return text

# remove speficific elements identified in data frame
def remove_elements(text, column):
    if column is not None:
        for element in column:
            text = text.replace(element, '')
    return text

# convert all entryies to lower case
def convert_to_lower_case(text):
    return text.lower()

# call the expand_short_forms function above on the tokenised words and expand into full forms
def expand_short_forms_in_text(text):
    words = text.split()
    expanded_words = [expand_short_forms(word, words[i-1]) if i > 0 else word for i, word in enumerate(words)]
    return ' '.join(expanded_words)

# remove special characters
def remove_special_characters(text):
    return re.sub(r'[^a-z0-9]', ' ', text)

# tokenizing text
def tokenize_text(text):
    return word_tokenize(text)

# less meaningful stop words removal using nltk
def remove_stop_words(words):
    stop_words = set(stopwords.words('english'))
    return (word for word in words if word not in stop_words)

# lemmatizing the words into base form using nltk
def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

# joining the words back
def combine_words(words):
    return ' '.join(word for word in words)

# remove spaces left in the removal proceses
def remove_extra_spaces(text):
    text = text.strip()
    return re.sub(r'\s+', ' ', text)

Firstly, the string <b>" &lt;br /&gt;" and ratings are removed</b> as identified in the word statistic analysis above. These also <b>removes the special characters</b>. The entires in the balanced data runs through iterations of cleaning process to <b>remove these strings</b>. On the other hand, the entries are also converted to <b>full forms</b> and in <b>small case.</b>

In [ ]:
# cleaning for specific strings, change to lower case full forms and remove special characters
def text_processing_pipeline_no_token(text, string=None, column=None):
    text = remove_string_entries(text, string)
    text = remove_elements(text, column)
    text = convert_to_lower_case(text)
    text = expand_short_forms_in_text(text)
    text = remove_special_characters(text)

    return text

In [ ]:
# make a copy of the balanced data
processed_data = balanced_data.copy()

# Iteratively run the processing pipeline on every entry of content
for i in range(len(balanced_data)):
    processed_data.loc[i, 'Content'] = text_processing_pipeline_no_token(
        processed_data.loc[i, 'Content'], "<br />", num_num_df['string_removal']
    )

In [ ]:
pos_lexical_diversity_part1 = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'pos']["Content"]))
neg_lexical_diversity_part1 = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'neg']["Content"]))

print("Lexical Diversity before cleaning (Positive Sentiment):", pos_lexical_diversity_part1)
print("Lexical Diversity before cleaning (Negative Sentiment):", neg_lexical_diversity_part1)

The lexical diversity <b>decresed by about 0.009</b> after this cleaning process.

Then, the string is tokenised for lemmatisation and stopwords removal.

In [ ]:
# Tokenise for lemmatisation to reduce word forms
def tpl_lemma(text):
    words = tokenize_text(text)
    words = lemmatize_words(words)
    processed_text = combine_words(words)
    processed_text = remove_extra_spaces(processed_text)
    return processed_text

In [ ]:
# Iteratively run the preprocessing on every entry of content
for i in range(len(processed_data)):
    processed_data.loc[i, 'Content'] = tpl_lemma(processed_data.loc[i, 'Content'])

In [ ]:
pos_lexical_diversity = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'pos']["Content"]))
neg_lexical_diversity = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'neg']["Content"]))

print("Lexical Diversity before cleaning (Positive Sentiment):", pos_lexical_diversity)
print("Lexical Diversity before cleaning (Negative Sentiment):", neg_lexical_diversity)

The lexical diversity <b>decresed by about 0.003</b> after lemmatization.

In [ ]:
# Tokenise for stopwords removal 
def tpl_stopword(text):
    words = tokenize_text(text)
    words = remove_stop_words(words)
    processed_text = combine_words(words)
    processed_text = remove_extra_spaces(processed_text)
    return processed_text

In [ ]:
# Iteratively run the preprocessing on every entry of content
for i in range(len(processed_data)):
    processed_data.loc[i, 'Content'] = tpl_stopword(processed_data.loc[i, 'Content'])

In [ ]:
pos_lexical_diversity = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'pos']["Content"]))
neg_lexical_diversity = calculate_lexical_diversity(' '.join(processed_data[processed_data['Sentiment'] == 'neg']["Content"]))

print("Lexical Diversity before cleaning (Positive Sentiment):", pos_lexical_diversity)
print("Lexical Diversity before cleaning (Negative Sentiment):", neg_lexical_diversity)

The lexical diversity <b>increased by about 0.026</b> after stopword removal. This is a irregular trend which the report will analys in detail later. Although the lexical diversity increased, the stopwords removal process is still needed as the analysis in the bar plot of the Top 30 common words above showed that <b>most of the commonly occured words are actually stop words</b>. Without removing stop words will <b>introduce a lot of noise</b> in the model later. 

The analysis for lexical diversity can be found in <a href="#2.1.3">2.1.3 Vocabulary statistic</a>. 

the following algorithm is used to check if the text has all being processed.

In [ ]:
# Check the success of text processing
success_count = 0
for original_text, processed_text in zip(balanced_data['Content'], processed_data['Content']):

    if original_text == processed_text:
        print("Text processing failed")
    else:
        success_count +=1
if success_count == len(processed_data['Content']):
    print("Text processing success")

<h3 id="2.1.3">2.1.3 Vocabulary statistic</h3>

<h4>lexical analysis:</h4>

This is interesting to see a significant and dramatic incrase in lexical diversity after stopwords removal. In most cases, the stopwards removal process <b>should decrease the lexical diversity</b>, however, <b>it increased there</b>. With reference to the equation below, lexical diversity is the number of unique words over the total number of words, a significant increase in lexical diversity after stopwords removal could mean that the original text contains <b>high proportion of stopwords as compared to total word count</b>. 

In this case, stop words removal process is carried out in short strings of text which is originated from <b>movie reviews</b>. The text of this from are generally short, which is likely to have a <b>higher proportion of stop words</b> as compared to long passage. 

This assumption is confirmed by the plot for top 30 common words in <a href="#2.1.3">section 2.1.3</a> which shows a significant propotion of stopwords.

$$
\text{Lexical Diversity} = \frac{\text{Number of Unique Words}}{\text{Total Number of Words}}
$$


<h4>frequency distribution</h4>

Checking the most common words and frequency distribution after tokenizing, lemmatizing and stopwords removal:

In [ ]:
# Separate the data based on sentiment for balanced data
positive_data = processed_data[processed_data['Sentiment'] == 'pos']
negative_data = processed_data[processed_data['Sentiment'] == 'neg']

In [ ]:
# Tokenize the words in the positive_data["Content"] column
positive_words = nltk.word_tokenize(' '.join(positive_data["Content"]))
positive_freq_dist = FreqDist(positive_words)
positive_common_words = positive_freq_dist.most_common(15)
positive_words, positive_frequencies = zip(*positive_common_words)

# Tokenize the words in the negative_data["Content"] column
negative_words = nltk.word_tokenize(' '.join(negative_data["Content"]))
negative_freq_dist = FreqDist(negative_words)
negative_common_words = negative_freq_dist.most_common(15)
negative_words, negative_frequencies = zip(*negative_common_words)

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot for positive sentiments
ax1.bar(positive_words, positive_frequencies)
ax1.set_xlabel('Words')
ax1.set_ylabel('Frequency')
ax1.set_title('Top 15 Most Common Words (Positive Sentiment)')
ax1.tick_params(axis='x', rotation=45)

# Plot for negative sentiments
ax2.bar(negative_words, negative_frequencies)
ax2.set_xlabel('Words')
ax2.set_ylabel('Frequency')
ax2.set_title('Top 15 Most Common Words (Negative Sentiment)')
ax2.tick_params(axis='x', rotation=45)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

From above, the most common occurance of words are film and movie. <b>Most</b> of the common words in both sentiment review dataset have a <b>neutral connotation</b>. For the top 15 most commonly used words, <b>two words</b> (good, great) in the positive sentiment data set have a <b>positive connotation</b> while there is only <b>one word</b> in the negative sentiment data set has a <b>negative connotation</b>. It is interesting to see that one word (good) with string positive connotation are present in the negative sentiment dataset. Assuming the dataset is labbeled correctly, this could be due to the <b>problem of negation</b>

The plot also confirms the result of stopwords removal and othere data cleaning technics. There is <b>no words</b> in shown in the graph that is a <b>stop word or tag and punctuation mark</b>. The words are also <b>all in lower case</b>.

A word cloud is generated to show the distribution of the words

In [ ]:
# Generate word cloud for positive sentiments
positive_wordcloud = WordCloud(width=800, height=400, background_color='white',colormap='inferno').generate(' '.join(positive_words))

# Generate word cloud for negative sentiments
negative_wordcloud = WordCloud(width=800, height=400, background_color='white',colormap='viridis').generate(' '.join(negative_words))

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot for positive sentiments
ax1.imshow(positive_wordcloud, interpolation='bilinear')
ax1.set_title('Word Cloud (Positive Sentiment)')
ax1.axis('off')

# Plot for negative sentiments
ax2.imshow(negative_wordcloud, interpolation='bilinear')
ax2.set_title('Word Cloud (Negative Sentiment)')
ax2.axis('off')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

<h4>Preparing data for Machine Learning:</h4>

The Machine Learning Task is to <b>predict the sentiment based on the input data in the form of text</b>.

Thus, the Content column is the <b>feature</b> to be learn and the Sentiment column is the <b>target variable/b>.

In [ ]:
# Predicting sentiment base on the content, thus X (features) is the content and y (target variable) is the sentiment
X = processed_data['Content']
y = processed_data['Sentiment']

80% of the data will be used for training while 20% of the data is stored for testing of the model.

In [ ]:
# Split data into training and testing data with a test size of 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

To achieve consistency, data is saved in the csv file

In [ ]:
# Save in testing and training data in a new dataframe
train_data = pd.DataFrame({'Content': X_train, 'Sentiment': y_train})
test_data = pd.DataFrame({'Content': X_test, 'Sentiment': y_test})
# Save data in csv file
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [ ]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

<h3 id="2.2">2.2 Baseline Performance</h3>

To Evaluate the 

Evaluation matrix:

In [ ]:
def evaluation(y_true, y_pred):
    # Calculate accuracy
    accuracy = (y_true == y_pred).mean()

    # Construct confusion matrix
    confusion_matrix = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'])

    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Calculate mean squared error
    mse = mean_squared_error(y_true, y_pred)

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("=" * 50)
    print("Confusion Matrix:\n", confusion_matrix)
    print("=" * 50)
    print("Precision:", precision)
    print("=" * 50)
    print("Recall:", recall)
    print("=" * 50)
    print("F1 Score:", f1)
    print("=" * 50)
    print("Mean Squared Error:", mse)

    return accuracy, confusion_matrix, precision, recall, f1, mse


Deploying the BERT model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('textattack/bert-base-uncased-imdb')
model = AutoModelForSequenceClassification.from_pretrained('textattack/bert-base-uncased-imdb')

In [ ]:
test_data = pd.read_csv('test_data.csv')
content_test = test_data['Content'].tolist()
sentiment_test = test_data['Sentiment'].tolist()

In [ ]:
# Initialize empty lists for predictions and evaluation metrics
predictions = []
y_true = []

In [ ]:
# Iterate over the test data
for content, sentiment in zip(content_test, sentiment_test):
    # Encode the content using the tokenizer
    tokens = tokenizer.encode_plus(content, add_special_tokens=True, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    prediction = outputs.logits.argmax(dim=1).item()

    # Append the prediction and true label to the lists
    predictions.append(prediction)

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Convert labels from strings to numerical values
sentiment_encoded = label_encoder.fit_transform(sentiment_test)

# Convert the lists to numpy arrays
y_pred = np.array(predictions)
y_true = np.array(sentiment_encoded)  # Use the encoded labels instead of sentiment_test

In [ ]:
# Call the evaluation function
accu_BERT, confuM_BERT, pre_BERT, recall_BERT, f1_BERT, mse_BERT = evaluation(y_true, y_pred)

<h3 id="2.3">2.3 Classification approach</h3>

Training and running the model RNN

stopwords removal - high tf is likely to be the stop words

In [ ]:
# Load the training data
train_data = pd.read_csv('train_data.csv')

# Extract the content and sentiment columns
content = train_data['Content'].tolist()
sentiment = train_data['Sentiment'].tolist()

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(content, sentiment, test_size=0.2, random_state=42)

In [ ]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=4000)

# Fit the vectorizer on the training data
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()

# Transform the validation data using the fitted vectorizer
X_val_tfidf = vectorizer.transform(X_val).toarray()

# Reshape the input data for LSTM
X_train_reshaped = X_train_tfidf.reshape(X_train_tfidf.shape[0], 1, X_train_tfidf.shape[1])
X_val_reshaped = X_val_tfidf.reshape(X_val_tfidf.shape[0], 1, X_val_tfidf.shape[1])

# Convert sentiment labels to numeric
sentiment_mapping = {'pos': 1, 'neg': 0}
y_train_numeric = np.array([sentiment_mapping[sent] for sent in y_train])
y_val_numeric = np.array([sentiment_mapping[sent] for sent in y_val])
y_test_numeric = np.array([sentiment_mapping[sent] for sent in sentiment_test])

In [ ]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.3, input_shape=(1, X_train_tfidf.shape[1])))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_numeric, validation_data=(X_val_reshaped, y_val_numeric),
          epochs=20, verbose=1, batch_size=128)

# Save the trained model
model.save("RNN_tf_idf.h5")

In [ ]:
# Load the trained model
model = load_model("RNN_tf_idf.h5")

# Load the test data
test_data = pd.read_csv('test_data.csv')

# Extract the content and sentiment columns
content_test = test_data['Content'].tolist()
sentiment_test = test_data['Sentiment'].tolist()

# Transform the test data using the fitted vectorizer
test_tfidf = vectorizer.transform(content_test).toarray()

# Reshape the input data for LSTM
test_reshaped = test_tfidf.reshape(test_tfidf.shape[0], 1, test_tfidf.shape[1])

# Evaluate the model on the test data
y_pred_prob = model.predict(test_reshaped)
y_pred = (y_pred_prob > 0.5).astype(int)
y_pred = y_pred.flatten()  # Flatten the predictions to match the shape of y_test_numeric

# Call the evaluation function
accu_RNN, confuM_RNN, prec_RNN, recall_RNN, f1_RNN, mse_RNN = evaluation(y_test_numeric, y_pred)

In [ ]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.3, 
               input_shape=(1, X_train_tfidf.shape[1])))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu', 
                kernel_regularizer=regularizers.l1(0.02)))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_numeric, validation_data=(X_val_reshaped, y_val_numeric),
          epochs=20, verbose=1, batch_size=128)

# Save the trained model
model.save("trained_model_RNN_tf_idf.h5")

In [ ]:
# Load the trained model
model = load_model("trained_model_RNN_tf_idf.h5")

# Load the test data
test_data = pd.read_csv('test_data.csv')

# Extract the content and sentiment columns
content_test = test_data['Content'].tolist()
sentiment_test = test_data['Sentiment'].tolist()

# Transform the test data using the fitted vectorizer
test_tfidf = vectorizer.transform(content_test).toarray()

# Reshape the input data for LSTM
test_reshaped = test_tfidf.reshape(test_tfidf.shape[0], 1, test_tfidf.shape[1])

# Evaluate the model on the test data
y_pred_prob = model.predict(test_reshaped)
y_pred = (y_pred_prob > 0.5).astype(int)
y_pred = y_pred.flatten()  # Flatten the predictions to match the shape of y_test_numeric

# Call the evaluation function
accuracy_RNN_tu, confuM_RNN_tu, precision_RNN_tu, recall_RNN_tu, f1_RNN_tu, mse_RNN_tu = evaluation(y_test_numeric, 
                                                                                     y_pred)


tf-idf measure the originality of the words. TF(t,d)*IDF(t) identify sigificant words. Number of times word appear/total words. 

SVM

In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the content data
X_train_vectorized = vectorizer.fit_transform(X_train)
X_val_vectorized = vectorizer.transform(X_val)

# Convert string labels to numeric format
label_encoder = LabelEncoder()
y_val_numeric = label_encoder.fit_transform(y_val)

In [ ]:
# Define a list of kernels to compare
kernels = ['linear', 'rbf', 'poly', 'sigmoid']

# Store the best performing kernel and its accuracy
best_kernel = None
best_accuracy = 0.0
best_modelSVM = None

# Iterate over each kernel
for kernel in kernels:
    # Train an SVM classifier with the current kernel
    model = SVC(kernel=kernel)
    model.fit(X_train_vectorized, y_train)

    # Make predictions on the validation set
    val_predictions = model.predict(X_val_vectorized)
    val_predictions_numeric = label_encoder.transform(val_predictions)

    # Calculate validation accuracy
    accuracy = accuracy_score(y_val_numeric, val_predictions_numeric)
    print("Validation Accuracy for", kernel, "kernel:", accuracy)

    # Check if the current kernel outperforms the previous best kernel
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_kernel = kernel
        best_modelSVM = model

# Fit the best model
best_modelSVM.fit(X_train_vectorized, y_train)
# Output the best performing kernel
print("Best Performing Kernel:", best_kernel)

In [ ]:
# Extract the content and sentiment columns
X_test = test_data['Content']
y_test = test_data['Sentiment']

# Vectorize the testing data
X_test_vectorized = vectorizer.transform(X_test)

# Make predictions on the testing set
test_predictions = best_modelSVM.predict(X_test_vectorized)
test_predictions_numeric = label_encoder.transform(test_predictions)

# Convert string labels of the testing set to numeric format
y_test_numeric = label_encoder.transform(y_test)

# Calculate accuracy on the testing set
accuracy_test = accuracy_score(y_test_numeric, test_predictions_numeric)

# Print the evaluation metrics for the testing set
accuracy_SVM, confuM_SVM, precision_SVM, recall_SVM, f1_SVM, mse_SVM = evaluation(y_test_numeric, test_predictions_numeric)

Logistic Regression (Bag of word)

In [ ]:
# Scale the vectorized data
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_vectorized)
X_val_scaled = scaler.transform(X_val_vectorized)

# Convert string labels to numeric format
label_encoder = LabelEncoder()
y_val_numeric = label_encoder.fit_transform(y_val)

In [ ]:
# Define the solvers to try
solvers = ['liblinear', 'sag', 'saga']

best_solver = None
best_accuracy = 0.0
best_modelLR = None


for solver in solvers:
    # Train a Logistic Regression classifier with the current solver
    model = LogisticRegression(max_iter=1000, solver=solver)
    model.fit(X_train_vectorized, y_train)
    
    # Make predictions on the validation set
    val_predictions = model.predict(X_val_vectorized)
    val_predictions_numeric = label_encoder.transform(val_predictions)

    # Calculate validation accuracy
    accuracy = accuracy_score(y_val_numeric, val_predictions_numeric)
    print("Validation Accuracy with", solver, "solver:", accuracy)
    
    # Check if the current solver achieved the best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_solver = solver
        best_modelLR = model
        
# Fit the best model
best_modelLR.fit(X_train_vectorized, y_train)
print("Best Solver:", best_solver)
print("Best Validation Accuracy:", best_accuracy)

In [ ]:
# Make predictions on the testing set
test_predictions = best_modelLR.predict(X_test_vectorized)
test_predictions_numeric = label_encoder.transform(test_predictions)

# Convert string labels of the testing set to numeric format
y_test_numeric = label_encoder.transform(y_test)

# Calculate accuracy on the testing set
accuracy_test = accuracy_score(y_test_numeric, test_predictions_numeric)

# Print the evaluation metrics for the testing set
accuracy_BoW, confuM_BoW, precision_BoW, recall_BoW, f1_BoW, mse_BoW = evaluation(y_test_numeric,
                                                                                            test_predictions_numeric)

<h2 id="Conc">3. Conclusion</h2>

<h3 id="3.1">3.1 Evaluation</h3>

In [ ]:
# Create a list of evaluation metrics
precision_scores = [precision_RNN, precision_RNN_tu, precision_SVM, precision_BoW, precision_BERT]
recall_scores = [recall_RNN, recall_RNN_tu, recall_SVM, recall_BoW, recall_BERT]
f1_scores = [f1_RNN, f1_RNN_tu, f1_SVM, f1_BoW, f1_BERT]
mse_scores = [mse_RNN, f1_RNN_tu, mse_SVM, mse_BoW, mse_BERT]

# create a list of model name
models = ['RNN (Overfitted)', 'RNN (L2 regularised)', 'SVM',
          'BoW', 'BERT (Baseline)']

# Set the figure size and create subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Set the width of each bar
bar_width = 0.4

# Set the position of each bar on the x-axis
r = np.arange(len(models))

# Initialize the bottom position of the bars
bottom = np.zeros(len(models))

# Plot precision scores
axs[0, 0].bar(r, precision_scores, bottom=bottom, color='b', width=bar_width)
axs[0, 0].bar(r[-1], precision_scores[-1], color='orange', width=bar_width)
axs[0, 0].axhline(y=precision_scores[-1], color='red', linestyle='-', linewidth=1)
axs[0, 0].set_ylim(0.8, 0.9)  
axs[0, 0].set_xlabel('Models')
axs[0, 0].set_xticks(r , models, rotation=300, ha='left', va='top')
axs[0, 0].set_ylabel('Precision')
axs[0, 0].set_title('Comparison of Precision Scores for Different Models')

# Plot recall scores
axs[0, 1].bar(r, recall_scores, bottom=bottom, color='b', width=bar_width)
axs[0, 1].bar(r[-1], recall_scores[-1], color='orange', width=bar_width)
axs[0, 1].axhline(y=recall_scores[-1], color='red', linestyle='-', linewidth=1)
axs[0, 1].set_ylim(0.8, 0.9)  # Adjust the limits
axs[0, 1].set_xlabel('Models')
axs[0, 1].set_xticks(r , models, rotation=300, ha='left', va='top')
axs[0, 1].set_ylabel('Recall')
axs[0, 1].set_title('Comparison of Recall Scores for Different Models')

# Plot F1 scores
axs[1, 0].bar(r, f1_scores, bottom=bottom, color='b', width=bar_width)
axs[1, 0].bar(r[-1], f1_scores[-1], color='orange', width=bar_width)
axs[1, 0].axhline(y=f1_scores[-1], color='red', linestyle='-', linewidth=1)
axs[1, 0].set_ylim(0.8, 0.9) 
axs[1, 0].set_xlabel('Models')
axs[1, 0].set_xticks(r , models, rotation=300, ha='left', va='top')
axs[1, 0].set_ylabel('F1-score')
axs[1, 0].set_title('Comparison of F1 Scores for Different Models')

# Plot MSE scores
axs[1, 1].bar(r, mse_scores, bottom=bottom, color='b', width=bar_width)
axs[1, 1].bar(r[-1], mse_scores[-1], color='orange', width=bar_width)
axs[1, 1].axhline(y=mse_scores[-1], color='red', linestyle='-', linewidth=1)
axs[1, 1].set_ylim(0, 1)  
axs[1, 1].set_xlabel('Models')
axs[1, 1].set_xticks(r , models, rotation=300, ha='left', va='top')
axs[1, 1].set_ylabel('MSE')
axs[1, 1].set_title('Comparison of MSE Scores for Different Models')

# Adjust the spacing between subplots
fig.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Create a list of accuracy scores
accuracy_scores = np.array([accuracy_RNN, accuracy_RNN_tu, accuracy_SVM, accuracy_BoW, accuracy_BERT])

plt.figure(figsize=(10, 6))

# Set the width of each bar
bar_width = 0.4

# Set the position of each bar on the x-axis
r = np.arange(len(models))

# Initialize the bottom position of the bars
bottom = np.zeros(len(models))

# Plot the accuracy scores with blue color below the red line
plt.bar(r, np.where(accuracy_scores > accuracy_scores[-1], accuracy_scores - accuracy_scores[-1], 0),
    bottom=accuracy_scores[-1], color='lightblue', width=bar_width)

# Plot the accuracy scores with light blue color above the red line
plt.bar(r, accuracy_scores, bottom=bottom, color='b', width=bar_width)

plt.bar(r[-1], accuracy_scores[-1], color='orange', width=bar_width)  # BERT plot in orange

# Draw a line at the top of the BERT bar
plt.axhline(y=accuracy_scores[-1], color='red', linestyle='-', linewidth=1)

# Set the y-axis limits to focus on the desired region
plt.ylim(0.8, 0.9)  # Adjust the limits

# Plot the accuracy scores with light blue color above the red line
plt.bar(r, np.where(accuracy_scores > accuracy_scores[-1], 
                    accuracy_scores - accuracy_scores[-1], 0), 
        bottom=accuracy_scores[-1],
        color='lightblue', width=bar_width)

# Add x-axis ticks and labels
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.xticks(r , models, rotation=300, ha='left', va='top')
plt.title('Comparison of Accuracy Scores for Different Models')

# Add legend
below = mpatches.Patch(color='b', label='Below Baseline Model')
above = mpatches.Patch(color='lightblue', label='Above Baseline Model')
base = mpatches.Patch(color='orange', label='Baseline Model(BERT)')
plt.legend(handles=[below, above,base])

# Display the plot
plt.show()

<h3 id="3.2">3.2 Project Evaluation and Summary</h3>